In [6]:
import os
import threading
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel, RandomForestClassifier
from pyspark.ml.pipeline import PipelineModel

from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit, sum, when
from pyspark.sql.types import StringType

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

24/05/09 18:39:49 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:39:49 WARN BlockManager: Block input-0-1715272789600 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:41:53 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:41:53 WARN BlockManager: Block input-0-1715272912800 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:42:55 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:42:55 WARN BlockManager: Block input-0-1715272974800 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:44:58 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:44:58 WARN BlockManager: Block input-0-1715273098000 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:46:01 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:46:01 WARN BlockManager: Block input-0-1715273161200 replicated to

In [2]:
# Change file path
file_path = "/Users/hydraze/Library/CloudStorage/GoogleDrive-tohziyu2@gmail.com/My Drive/Studies/KU Leuven/Courses/Classes/Y1S2/Advanced Analytics in Business/Project/3/AdvancedAnalytics_Streaming-Text-Analytics/"
os.chdir(file_path)


In [3]:
# read pickled model via pipeline api
mPath =  file_path+"models/best_model"
best_model = PipelineModel.load(mPath)

In [4]:
# Toy predict function that returns a random probability. Normally you'd use your loaded globals()['my_model'] here
def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))

    # Convert to data frame
    df = spark.read.json(rdd)

    # Data cleaning (to update concurrently with the other jupyter notebook)
    
    # Extracting type of post: Show HN
    df = df.withColumn('isShowHN', when(df.title.contains("Show HN"), 1).otherwise(0))
    
    # Extracting time of day
    extract_time_of_day_udf = udf(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%H'))
    
    df = df.withColumn('time_of_day', extract_time_of_day_udf(df.posted_at))
    
    # Extracting day of week
    weekDay =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%w'))
    
    df = df.withColumn('day_of_week', weekDay(df.posted_at))
    
    # And then predict using the loaded model (uncomment below):
    df_result = best_model.transform(df)
    df_result.show()

In [5]:
# Likely the usual streaming
ssc = StreamingContext(sc, 10)
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)
ssc_t = StreamingThread(ssc)
ssc_t.start()

24/05/09 18:20:52 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:20:52 WARN BlockManager: Block input-0-1715271652000 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:20:55 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:20:55 WARN BlockManager: Block input-0-1715271655000 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:20:58 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:20:58 WARN BlockManager: Block input-0-1715271658000 replicated to only 0 peer(s) instead of 1 peers


========= 2024-05-09 18:21:00 =========


24/05/09 18:21:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-------+-----+--------+-----------+-----------+--------------+---------------------+--------------+-------------+--------------------+--------------------+--------------------+----------+
|     aid|comments|              domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url|   user|votes|isShowHN|time_of_day|day_of_week|num_col_vector|scaled_num_col_vector|isShowHN_index| isShowHN_OHE|final_feature_vector|       rawPrediction|         probability|prediction|
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-------+-----+--------+-----------+-----------+--------------+---------------------+--------------+-------------+--------------------+--------------------+-------

24/05/09 18:21:02 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:21:02 WARN BlockManager: Block input-0-1715271662000 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:21:04 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:21:04 WARN BlockManager: Block input-0-1715271664000 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:21:06 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:21:06 WARN BlockManager: Block input-0-1715271666200 replicated to only 0 peer(s) instead of 1 peers


...

24/05/09 18:35:39 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:35:39 WARN BlockManager: Block input-0-1715272539200 replicated to only 0 peer(s) instead of 1 peers


========= 2024-05-09 18:35:40 =========
+--------+--------+-------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+---------+-----+--------+-----------+-----------+--------------+---------------------+--------------+-------------+--------------------+--------------------+--------------------+----------+
|     aid|comments|       domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url|     user|votes|isShowHN|time_of_day|day_of_week|num_col_vector|scaled_num_col_vector|isShowHN_index| isShowHN_OHE|final_feature_vector|       rawPrediction|         probability|prediction|
+--------+--------+-------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+---------+-----+--------+-----------+-----------+--------------+---------------------+--------------+-------------+--------------------+---

24/05/09 18:37:44 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:37:44 WARN BlockManager: Block input-0-1715272664400 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:37:46 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:37:46 WARN BlockManager: Block input-0-1715272666400 replicated to only 0 peer(s) instead of 1 peers


========= 2024-05-09 18:37:50 =========
+--------+--------+-----------+---------+-------------------+--------------------+----------------+--------------------+--------------------+---------+-----+--------+-----------+-----------+--------------+---------------------+--------------+-------------+--------------------+--------------------+--------------------+----------+
|     aid|comments|     domain|frontpage|          posted_at|         source_text|    source_title|               title|                 url|     user|votes|isShowHN|time_of_day|day_of_week|num_col_vector|scaled_num_col_vector|isShowHN_index| isShowHN_OHE|final_feature_vector|       rawPrediction|         probability|prediction|
+--------+--------+-----------+---------+-------------------+--------------------+----------------+--------------------+--------------------+---------+-----+--------+-----------+-----------+--------------+---------------------+--------------+-------------+--------------------+--------------------+

========= 2024-05-09 18:39:50 =========
+--------+--------+---------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-----+-----+--------+-----------+-----------+--------------+---------------------+--------------+-------------+--------------------+--------------------+--------------------+----------+
|     aid|comments|         domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url| user|votes|isShowHN|time_of_day|day_of_week|num_col_vector|scaled_num_col_vector|isShowHN_index| isShowHN_OHE|final_feature_vector|       rawPrediction|         probability|prediction|
+--------+--------+---------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-----+-----+--------+-----------+-----------+--------------+---------------------+--------------+-------------+--------------------+---------

In [9]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


24/05/09 18:12:05 WARN StreamingContext: StreamingContext has not been started yet
24/05/09 18:12:07 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/09 18:12:07 WARN BlockManager: Block input-0-1715271127000 replicated to only 0 peer(s) instead of 1 peers
24/05/09 18:12:10 ERROR JobScheduler: Error running job streaming job 1715271130000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/hydraze/Downloads/spark/spark-3.5.1-bin-hadoop3/python/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
        ^^^^^^^^^^^^^^^^^^^
  File "/var/folders/4h/j84nnmnn0nj01mk0ndxyk7n40000gn/T/ipykernel_54142/3537873331.py", line 10, in process
    df = spark.createDataFrame(rdd)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hydraze/Downloads/spark/spark-3.5.1-bin-hadoop3/python/pyspark/sql/session.py", line 1443, in createDataFrame
    return self._create_dataframe(
      